In [68]:
import pandas as pd
import locale


In [69]:
# Load the mapping table
mapping_table = {
    'Cuota Socio Anual': 1,
    'Cuota Socio Mensual': 23,
    'Cuota Socio Trimestral': 36,
    'Donación Particular': 7,
    'Donación Empresa': 22,
    'Subvención Pública': 6,
    'Subvención Privada': 11,
    'Telefonía': 29,
    'Electricidad': 29,
    'Agua': 29,
    'Gastos Oficina Valencia': 29,
    'Gastos Oficina Madrid': 29,
    'Limpieza': 30,
    'Asesoramiento juridico': 31,
    'Gestoría': 31,
    'Mantenimiento Web': 32,
    'Gastos tecnología': 32,
    'Mensajería': 33,
    'Comisiones': 34,
    'Póliza de seguro': 35,
    'Prevención de Riesgos': 35,
    'Servicio Psicología Grupal': 35,
    'Pago Factura Emitida': 35,
    'Pago sin Factura': 35,
    'Corona Flores': 35,
    'Fichaje Horario': 35,
    'Teaming': 35,
    'Merchandising varios': 2,
    'Bodas, comuniones y bautizos': 12,
    'Calendario': 24,
    'Eventos': 25,
    'Cursos formación': 14,
    'Seguridad Social': 18,
    'Membresia': 19,
    'Nómina': 20,
    'Liquidación Finiquito': 20,
    'Modelo 111': 26,
    'Modelo 303': 26,
    'Becas AESR': 9,
    'Encuentro familias': 27,
    'Cumpleaños niñas': 17,
    'Finrett': 17,
    'Comunicadores': 28,
    'Captación de fondos': 4,
    'Captación de fondos': 5,
    'Captación de fondos': 15,
    'Captación de fondos': 16,
    'Lotería': 21
}



In [70]:
# Load the transactions CSV file
transactions_df = pd.read_csv('../data/raw/transactions.csv', delimiter=';')

# Define a function to map TransactionCategory to transaction_category_id
def map_transaction_category(transaction_category):
    return mapping_table.get(transaction_category, None)

# Apply the mapping function to the transactions DataFrame
transactions_df['transaction_category_id'] = transactions_df['TransactionCategory'].apply(map_transaction_category)

# Convert transaction_category_id to integer type
transactions_df['transaction_category_id'] = transactions_df['transaction_category_id'].astype('Int64')



In [71]:
# Define a function to create the 'comment' field and adjust 'transaction_contact_id'
def create_comment_and_contact_id(contact_id):
    if isinstance(contact_id, str) and not contact_id.startswith("003"):
        return contact_id, None
    return None, contact_id

# Apply the function to create 'comment' and adjust 'transaction_contact_id'
transactions_df['comment'], transactions_df['transaction_contact_id'] = zip(*transactions_df['ContactID'].apply(create_comment_and_contact_id))

# Function to convert European formatted string to float
def parse_european_number(number):
    try:
        return locale.atof(number)
    except ValueError:
        return 0

# Apply the function to the Amount column and log the conversion
def convert_and_log_amount(amount):
    original_amount = amount
    converted_amount = parse_european_number(str(amount).replace(',', '.').strip())
    return converted_amount

transactions_df['Amount'] = transactions_df['Amount'].apply(convert_and_log_amount)

# Ensure transaction_amount is a positive integer by keeping the integer part intact
transactions_df['transaction_amount'] = transactions_df['Amount'].apply(lambda x: float(abs(x)))

# Create the output DataFrame with the required columns
output_df = pd.DataFrame({
    'transaction_amount': transactions_df['transaction_amount'],
    'transaction_contact_id': transactions_df['transaction_contact_id'],
    'transaction_date': transactions_df['TransactionDate'],
    'transaction_reference': transactions_df['Reference'],
    'transaction_category': transactions_df['transaction_category_id'],
    'transaction_detail': transactions_df['Detail'],
    'transaction_origin': 2,
    'soft_delete': False,
    'last_changed_date': None,
    'last_changed_by': None,
    'comment': transactions_df['comment'],
    'original_transaction_id': transactions_df['TransactionID']
})

# Save the updated transactions to a new CSV file with a custom delimiter
output_df.to_csv('../data/raw/updated_transactions.csv', index=False, sep=',')